# Human Facial Expression Recognition.
#Explore and compare artificial intelligence and machine learning techniques to understand accuracy in classifying human facial expressions from images.

# Loading All library 

In [ ]:
from os import listdir
from os.path import isfile, join
import numpy
import cv2
import numpy as np
from keras.models import Model, Sequential
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.applications.vgg19 import VGG19
from keras.layers import Dense, GlobalAveragePooling2D, Flatten
from keras.models import Model, Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from keras.applications.vgg16 import preprocess_input
from sklearn.neural_network import MLPClassifier
import seaborn as sns
import matplotlib.pyplot as plt
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from sklearn.metrics import classification_report,confusion_matrix

# Data Preprocessing and Loading All Images

In [ ]:
path="emotional data/"
file_anger_sad = path + "anger_sad/"
file_casual = path + "casual/"
file_joy_happy = path + "joy_happy/"
file_sorrow = path + "sorrow/"
file_surprise = path + "surprise/"
from os import listdir
import cv2

def loadImages(path):
    imagesList = listdir(path)
    loadedImages = []
    for image in imagesList:
        img = cv2.imread(path + image)
        img = cv2.resize(img, (50, 50))

        loadedImages.append(img)

    return loadedImages


file_anger_sad=loadImages(file_anger_sad)
file_casual = loadImages(file_casual)
file_joy_happy = loadImages(file_joy_happy)
file_sorrow = loadImages(file_sorrow)
file_surprise = loadImages(file_surprise)


total_X =[]
label= []


for i in range(len(file_anger_sad)):
    total_X.append(file_anger_sad[i])
    label.append("0")

for v in range(len(file_casual)):
    total_X.append(file_casual[v])
    label.append("1")

for n in range(len(file_joy_happy)):
    total_X.append(file_joy_happy[n])
    label.append("2")

for l in range (len (file_sorrow)):
    total_X.append (file_sorrow[l])
    label.append ("3")

for a in range(len(file_surprise)):
    total_X.append(file_surprise[a])
    label.append("4")


total_X = np.asarray(total_X)
label = np.asarray(label)


label = to_categorical(label)

# Splitting Data into Training and Testing 90% for Training And 10% for Testing

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(total_X, label, test_size=0.1, random_state=66 )


# VGG16 model, with weights pre-trained on ImageNet.
#The default input size for this model is 224x224.
#This model can be built both with 'channels_first' data format (channels, height, width) or 'channels_last' data format (height, width, channels).
Please Check keras webpage https://keras.io/applications/#vgg16


In [ ]:
base_model = VGG16 (weights='vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5', include_top=False,)
x = base_model.output
x = GlobalAveragePooling2D () (x)



x = Dense (100, activation='relu') (x)
x = Dropout (0.15) (x)
x = Dense (60, activation='relu') (x)
x = Dense (30, activation='relu') (x)

predictions = Dense(5, activation='softmax')(x)
model = Model(input=base_model.input, output=predictions)

for layer in base_model.layers:
    layer.trainable = False
model = Model(input=base_model.input, output=predictions)
model.compile (loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])


#Fitting Model 

In [ ]:
history=model.fit (X_train, y_train, epochs=50, batch_size=50, verbose=1,validation_data=(X_test, y_test))


#Model Summary

In [ ]:
model.summary()


# Ploting Model Performance 

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for loss


In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
import pandas as pd
#ROC Curve
yp_dum=pd.get_dummies(y_true)
yt_dum=pd.get_dummies(pred)

# Plot all ROC curves
# Compute ROC curve and ROC area for each class

In [ ]:
from sklearn.metrics import roc_curve, auc
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(len(class_names)):
    fpr[i], tpr[i], _ = roc_curve(yt_dum[i], yp_dum[i])
    roc_auc[i] = auc(fpr[i], tpr[i])

plt.figure()
colors=["blue","darkviolet","magenta","orange","gold"]
emote_list=np.unique(np.array(total_X))
for i, color, emotion in zip(range(len(class_names)), colors, emote_list):
    plt.plot(fpr[i], tpr[i], color=color,
             label='ROC curve of class %s (area = %.2f)' %(emotion, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([-.05, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Some extension of Receiver operating characteristic to multi-class')
plt.legend(loc="lower right")
plt.show()

In [ ]:
class_names=np.unique(np.array(CM)) #Used for Confusion Matrix visual.


In [ ]:
import scikitplot as skplt
skplt.metrics.plot_confusion_matrix(y_true, pred, normalize=True)
plt.show ()